In [61]:
import requests
import os
import json
import pandas as pd
import pprint

from dotenv import load_dotenv

In [62]:
 # Load environment variables from .env file
load_dotenv()

# Get the API key
api_key = os.getenv('NOAA_CLIMATE_DATA')
token = api_key

In [63]:
def get_weather_data(token, location_id, start_date, end_date, data_type):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    headers = {
        'token': token
    }
    params = {
        'datasetid': 'GSOY',  # Global Summary of the Month
        'locationid': location_id,  # Location ID for the city
        'startdate': start_date,
        'enddate': end_date,
        'datatypeid': data_type,
        'units': 'standard',  # Use 'standard' for Fahrenheit, inches, etc.
        'limit': 1000  # Adjust as needed
    }

    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        return response.status_code, response.text

In [64]:

# Example usage
location_id = "FIPS:AG"  # FIPS code for Albania
start_date = '1995-01-01'
end_date = '2000-12-31'
data_type = ["DP10", "DP1X", "DT32", "DX70", "DX90", "PRCP", "RHAV", "TAVG", 'TMAX']  # Example data type (Average Temperature)

weather_data = get_weather_data(token, location_id, start_date, end_date, data_type)

# Check if the response is successful or not
if isinstance(weather_data, tuple):
    # Error case
    print("Error occurred: Status Code:", weather_data[0], "Message:", weather_data[1])
else:
    # Success case
    # Parse the JSON strings into dictionaries
    metadata = weather_data["metadata"]
    results = weather_data["results"]

    # Create an empty DataFrame
    df = pd.DataFrame()

    # Iterate over each result
    for result in results:
        date = result["date"][:10]  # Extract date portion
        datatype = result["datatype"]
        value = result["value"]
        station = result["station"]
        
        # Check if the date already exists in the DataFrame
        if date not in df.index:
            df.loc[date, "station"] = station
        
        # Add the value to the corresponding cell
        df.loc[date, datatype] = value

    # Reset index to make date a column
    df.reset_index(inplace=True)
    # Rename the 'index' column to 'date'
    df.rename(columns={"index": "date"}, inplace=True)

    print(df)
    
    # Constructing filename
filename = f"{location_id}_{start_date}_{end_date}_weather_data.csv"
        
# Export the final dataframe to a CSV file with the constructed filename
df.to_csv(filename, index=False)

         date            station  DP10  DP1X  DT32   DX70   DX90   PRCP  TAVG  \
0  1995-01-01  GHCND:AG000060390   3.0   0.0  12.0  238.0  122.0   1.61  71.5   
1  1996-01-01  GHCND:AG000060390   8.0   0.0   0.0  342.0  149.0   2.36  72.0   
2  1997-01-01  GHCND:AG000060390  26.0   1.0   0.0  285.0  160.0  16.37  71.6   
3  1998-01-01  GHCND:AG000060390   5.0   0.0   1.0  332.0  176.0   1.30  72.8   

   TMAX  
0  81.3  
1  84.8  
2  89.4  
3  85.3  
